# Bulk CSV Upload
A notebook to read csvs and upload their data as [hosted feature layers](https://doc.arcgis.com/en/arcgis-online/share-maps/hosted-web-layers.htm) in ArcGIS

In [1]:
# common imports
import os
import pandas as pd
from arcgis.gis import GIS

***Note**, if you are unable to import local `partnerutils`, **copy the following functions** from [`feature_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/feature_utils.py)

In [2]:
from partnerutils.feature_utils import sdf_from_xyz

## User Input

* **gis**: your GIS instance, parameter information [here](https://developers.arcgis.com/python/guide/using-the-gis/)
* **dir_path**: path to directory with the CSVs
* **coord_fields**: map Point fields to CSV columns

In [4]:
# this will prompt for a password
# can also do GIS("https://www.arcgis.com", "<USERNAME>", "<PASSWORD>")
gis = GIS(username="mpayson_startups")

Enter password:  ········


In [5]:
dir_path = "../sample_data"
coord_fields = {
    "x": "Longitude",
    "y": "Latitude",
    #"z": "<Z COLUMN (optional)>"
}

## Execution

In [6]:
# create a new folder in the GIS to store the layers
folder = os.path.basename(dir_path)
gis_folder = gis.content.create_folder(folder)

# handle folder already exists
gis_folder = folder if gis_folder is None else gis_folder

In [7]:
# get an array of all the csvs in the directory
csvs = [file for file in os.listdir(dir_path) if file.endswith('.csv')]
csv_paths = [os.path.join(dir_path, csv) for csv in csvs]
n_paths = len(csv_paths)
print("Pushing {0} csvs".format(n_paths))

Pushing 1 csvs


In [8]:
# loop through csvs, build spatial dataframe, upload layers to GIS, move to new folder
z_field = coord_fields['z'] if 'z' in coord_fields else None
items = []
for i, csv_path in enumerate(csv_paths):
    df = pd.read_csv(csv_path)
    sdf = sdf_from_xyz(df, coord_fields['x'], coord_fields['y'], z_field)
    
    title = os.path.splitext(os.path.basename(csv_path))[0]
    lyr_item = gis.content.import_data(sdf, title=title, tags="esri_partner_tools_sample")
    lyr_item.move(gis_folder)
    
    items.append(lyr_item)
    print("{0}/{1}".format(i + 1, n_paths))

1/1


## Clean Up

In [15]:
delete_items = gis.content.search("tags:esri_partner_tools_sample")
gis.content.delete_items(delete_items)
gis.content.delete_folder(gis_folder['title'])

True